In [1]:
import numpy as cp
import matplotlib.pyplot as plt
from numpy import log as ln
import random
from numpy import random
import math
from matplotlib.pyplot import plot,savefig
import scipy
from scipy import signal
def relu(y):
    tmp = y.copy()
    tmp[tmp < 0] = 0
    return tmp
def drelu(x):
    tmp = x.copy()
    tmp[tmp >= 0] = 1
    tmp[tmp < 0] = 0
    return tmp

def softmax(y):
    y = y - cp.array(y.max(axis=0),ndmin=2)
    exp_y = cp.exp(y) 
    sumofexp = cp.array(exp_y.sum(axis=0),ndmin=2)
    softmax = exp_y/sumofexp
    return softmax

def divi_(lr0,global_step,decay_step):
    return lr0*(0.5**((int(global_step/decay_step))))
import sys
filePath = 'Utils/nup'
sys.path.append(filePath)
import load_MNIST
num =60000
mnist=(load_MNIST.load_mnist(one_hot=True))
train_data0 = mnist[0][0][0:num].reshape(num,28,28,1)
train_label0 = mnist[0][1][0:num].reshape(num,10,1)
test_data0 = mnist[1][0][0:10000].reshape(10000,28,28,1)
test_label0 = mnist[1][1][0:10000].reshape(10000,10,1)
def uni_permu(a,b,direction):
    if direction ==1:
        p = cp.random.permutation(len(a.T))
        return cp.array((a.T[p]).T), cp.array((b.T[p]).T)
    if direction == 0:
        p = cp.random.permutation(len(a))
        return cp.array((a[p])), cp.array((b[p]))
def mini_batch_generate(mini_batch_size,data1,label1):
    data = cp.array(data1*1)
    label = cp.array(label1*1)
    if (data.shape[1]%mini_batch_size == 0):
        n=data.shape[1]
    else:
        n = (int(data.shape[1]/mini_batch_size))*mini_batch_size
    data,label = uni_permu(data,label,1)
    mini_batches = cp.array([data[:,k:k+mini_batch_size] for k in range(0,n,mini_batch_size)])
    mini_batches_labels =cp.array([label[:,k:k+mini_batch_size] for k in range(0,n,mini_batch_size)])
    return (mini_batches),(mini_batches_labels)
def o_sigmoid(mat):
    return (1.0/(1+pow(cp.e,-mat))).reshape(mat.shape)
def sigmoid(mat):
    return (2.0/(1+pow(cp.e,-mat))-1.0).reshape(mat.shape)
def dsigmoid(mat):
    return (o_sigmoid(mat)*(1-o_sigmoid(mat)))
def padding(image, zero_num):
    if len(image.shape) == 4:
        image_padding = cp.zeros((image.shape[0],image.shape[1]+2*zero_num,image.shape[2]+2*zero_num,image.shape[3]))
        image_padding[:,zero_num:image.shape[1]+zero_num,zero_num:image.shape[2]+zero_num,:] = image
        return image_padding
    elif len(image.shape) == 3:
        image_padding = cp.zeros((image.shape[0]+2*zero_num, image.shape[1]+2*zero_num, image.shape[2]))
        image_padding[zero_num:image.shape[0]+zero_num, zero_num:image.shape[1]+zero_num,:] = image
        return image_padding
    else:
        print("维度错误")
        return 0
    


def pool(feature, size=2, stride=2):
    num,feature_h, feature_w, feature_ch = feature.shape
    pool_h = cp.uint16((feature_h - size)/stride + 1)
    pool_w = cp.uint16((feature_w - size)/stride + 1)
    feature_reshaped = feature.reshape(num,pool_h, feature_h//pool_h, pool_w, feature_w//pool_w, feature_ch)
    out = feature_reshaped.max(axis=2).max(axis=3)
    out_location_c = feature_reshaped.max(axis=2).argmax(axis=3)
    out_location_r = feature_reshaped.max(axis=4).argmax(axis=2)
    out_location = out_location_r * size + out_location_c
    return out, out_location

def rotate180(kernel, axis=(-2, -3)):
    return cp.flip(kernel, axis)
def add_bias(conv, bias):
    if conv.shape[-1] != bias.shape[0]:
        print("给卷积添加偏置维度出错")
    else:
        for i in range(bias.shape[0]):
            conv[:,:,:,i] += bias[i,0]
    return conv
def uni_permu(a1,b1,direction):
    a=a1*1
    b=b1*1
    if direction ==1:
        p = cp.random.permutation(len(a.T))
        return cp.array((a.T[p]).T), cp.array((b.T[p]).T)
    if direction == 0:
        p = cp.random.permutation(len(a))
        return cp.array((a[p])), cp.array((b[p]))
def mini_batch_generate(mini_batch_size,data1,label1):
    data = cp.array(data1*1).reshape(data1.shape[0],data1.shape[1]*data1.shape[2])
    label = cp.array(label1*1).reshape(label1.shape[0],label1.shape[1]*label1.shape[2])
    if (data.shape[0]%mini_batch_size == 0):
        n=data.shape[0]
    else:
        n = (int(data.shape[0]/mini_batch_size))*mini_batch_size
    data,label = uni_permu(data,label,0)
    mini_batches = cp.array([data[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)])
    mini_batches_labels =cp.array([label[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)])
    mini_batches = mini_batches.reshape(int(data.shape[0]/mini_batch_size),mini_batch_size,data1.shape[1],data1.shape[2],1)
    mini_batches_labels = mini_batches_labels.reshape(int(label.shape[0]/mini_batch_size),mini_batch_size,label.shape[1],1)
    return (mini_batches),(mini_batches_labels)
# def conv_(img, conv_filter):
#     # 对二维图像以及二维卷积核进行卷积，不填充后对图像数平均
#     num,img_h, img_w = img.shape
#     num,filter_h, filter_w = conv_filter.shape
#     feature_h = img_h - filter_h + 1
#     feature_w = img_w - filter_w + 1
#     img_matrix = cp.zeros((feature_h*feature_w, num*filter_h*filter_w))
#     img_o=0
#     for i in range(feature_h*feature_w):
#         img_matrix[i,:] = img[:,cp.uint16(i/feature_w):cp.uint16(i/feature_w+filter_h),cp.uint16(i%feature_w):cp.uint16(i%feature_w+filter_w)].reshape(num*filter_w*filter_h)        
#         filter_matrix = conv_filter.reshape(num*filter_h*filter_w,1)
#         img_out = cp.dot(img_matrix, filter_matrix)
#         img_out = img_out.reshape(feature_h,feature_w) 
#     return img_out/img.shape[0]

# def conv_(img, conv_filter):
#     # 对二维图像以及二维卷积核进行卷积，不填充后对图像数平均
#     num,img_h, img_w = img.shape
#     num,filter_h, filter_w = conv_filter.shape
#     feature_h = img_h - filter_h + 1
#     feature_w = img_w - filter_w + 1
#     img_out =[]
#     for i in range(num):
#         img_out.append(scipy.signal.convolve2d(image[i], np.fliplr(np.flipud(conv_filter[i])),
#                               mode = "valid")*1)
#     return np.average(img_out,axis=0)

def conv_cal_w(out_img_delta, in_img):
    #由卷积前的图片以及卷积后的delta计算卷积核的梯度
    #out:(500, 10, 10, 16),in_img: (500, 14, 14, 6)
    nabla_conv = cp.zeros([out_img_delta.shape[-1], in_img.shape[1]-out_img_delta.shape[1]+1, 
                           in_img.shape[2]-out_img_delta.shape[2]+1, in_img.shape[-1]])
    for filter_num in range(nabla_conv.shape[0]):
        for ch_num in range(nabla_conv.shape[-1]):
            nabla_conv[filter_num,:,:,ch_num] = conv_(in_img[:,:,:,ch_num], out_img_delta[:,:,:,filter_num])
    return nabla_conv

def conv_cal_b(out_img_delta):
    #shape: 500, 10, 10, 16
    return cp.average(out_img_delta,axis = (0,1,2)).reshape(out_img_delta.shape[-1],1)

        
def pool_delta_error_bp(pool_out_delta1, pool_out_max_location1, size=2, stride=2):
    pool_out_delta = pool_out_delta1*1
    pool_out_max_location = pool_out_max_location1*1
    num,pool_h, pool_w, pool_ch = pool_out_delta.shape
    in_h = cp.uint16((pool_h-1)*stride+size)
    in_w = cp.uint16((pool_w-1)*stride+size)
    in_ch = pool_ch
    #复原尺寸
    pool_out_delta_reshaped = pool_out_delta.transpose(0,3,1,2)#16*5*5
    pool_out_delta_reshaped = pool_out_delta_reshaped.reshape(num*pool_h*pool_w*pool_ch)
    
    pool_out_max_location_reshaped = pool_out_max_location.transpose(0,3,1,2)
    pool_out_max_location_reshaped = pool_out_max_location_reshaped.reshape(num*pool_h*pool_w*pool_ch)
    
    in_delta_matrix = cp.zeros([num*pool_h*pool_w*pool_ch,size*size])
    
    in_delta_matrix[cp.arange(num*pool_h*pool_w*pool_ch), pool_out_max_location_reshaped] = pool_out_delta_reshaped
    
    in_delta = in_delta_matrix.reshape(num,pool_ch,pool_h, pool_w, size, size)
    in_delta = in_delta.transpose(0,2,4,3,5,1)
    in_delta = in_delta.reshape(num,in_h, in_w, in_ch)
    return  in_delta

train_image = padding(train_data0,2)#对初始图像进行零填充，保证与LeNet输入结构一致60000*32*32*1
test_image = padding(test_data0,2)

def split_by_strides2(X, kh, kw, s):
    N, H, W, C = X.shape
    oh = (H - kh) // s + 1
    ow = (W - kw) // s + 1
    shape = (N, oh, ow, kh, kw, C)
    strides = (X.strides[0], X.strides[1]*s, X.strides[2]*s, *X.strides[1:])
    A = cp.lib.stride_tricks.as_strided(X, shape=shape, strides=strides)
    return A

def split_by_strides(x, kernel_size, stride=(1, 1)):
    """
    将张量按卷积核尺寸与步长进行分割
    :param x: 被卷积的张量
    :param kernel_size: 卷积核的长宽
    :param stride: 步长
    :return: y: 按卷积步骤展开后的矩阵
    """
    from numpy.lib.stride_tricks import as_strided
    B,C, h, w = x.shape
    out_H, out_W = (h - kernel_size[0]) // stride[0] + 1, (w - kernel_size[1]) // stride[1] + 1
    shape = (B,C, out_H, out_W, kernel_size[0], kernel_size[1])
    strides = (*x.strides[:-2], x.strides[-2] * stride[0],
               x.strides[-1] * stride[1], *x.strides[-2:])
    y = as_strided(x, shape, strides=strides)
    return y

def conv(img, conv_filter):
   
    if len(img.shape)!=4 or len(conv_filter.shape)!=4:
        print("卷积运算所输入的维度不符合要求")
        return 0
        
    if img.shape[-1] != conv_filter.shape[-1]:
        print("卷积输入图片与卷积核的通道数不一致")
        return 0
    img_2d = cp.copy(img)
    img_2d = img_2d.transpose(0,3,1,2)
    x_stride = split_by_strides(img_2d, conv_filter.shape[-3:-1], (1,1))*1
    filter2 = conv_filter.transpose(0,3,1,2)
    img_out = cp.tensordot(x_stride, filter2, axes=[(1, 4, 5), (1, 2, 3)]).transpose((0, 3, 1, 2))

    return img_out.transpose(0,2,3,1)



def conv2(img, conv_filter):
   
    if len(img.shape)!=4 or len(conv_filter.shape)!=4:
        print("卷积运算所输入的维度不符合要求")
        return 0
        
    if img.shape[-1] != conv_filter.shape[-1]:
        print("卷积输入图片与卷积核的通道数不一致")
        return 0
    img_num,img_h, img_w, img_ch = img.shape
    filter_num, filter_h, filter_w, img_ch = conv_filter.shape
    feature_h = img_h - filter_h + 1
    feature_w = img_w - filter_w + 1
    img_2d = cp.copy(img)   
    x_stride = split_by_strides2(img_2d, filter_h, filter_w, 1)*1
    filter2 = conv_filter.transpose(1,2,3,0)
    img_out = cp.tensordot(x_stride, filter2, axes=[(3,4,5), (0,1,2)]) 

    return img_out
def reverse_conv2d(x1, kernel1, rotate=False, invert=False):
    """
    conv2d的反向卷积,求梯度时用的
    @param x: 被卷积的张量
    @param kernel: 卷积核
    @param rotate: 卷积核旋转180度
    @param invert: 该参数有点迷,不好解释,简单的说就是反向卷积有两种,视卷积结果的形状需要调整一些轴的位置
    @return: 反向卷积结果
    """
    kernel = kernel1.transpose(0,3,1,2)
    x = x1.transpose(0,3,1,2)
    ksize = kernel.shape
    x = split_by_strides(x, ksize[-2:])
    if rotate:
        kernel = rotate180(kernel)
    i = 0 if invert else 1
    y = cp.tensordot(x, kernel, [(i, 4, 5), (0, 2, 3)])
    if invert:
        return y.transpose((3, 1, 2,0))/x1.shape[0]
    return y.transpose((0, 3, 1, 2))

In [2]:
class Adam:
    def __init__(self,theta):
        self.lr=0.01
        self.beta1=0.9
        self.beta2=0.999
        self.epislon=1e-8
        self.m=[cp.zeros(ms.shape) for ms in theta]
        self.s=[cp.zeros(ms.shape) for ms in theta]
        self.t=0
    
    def New_theta(self,theta,gradient,eta):
        self.t += 1
        self.lr = eta*1
        self.decay=1e-4
        g=gradient*1
        theta2 = [cp.zeros(ms.shape) for ms in theta]
        for l in range(len(gradient)):
            self.m[l] = self.beta1*self.m[l] + (1-self.beta1)*g[l]
            self.s[l] = self.beta2*self.s[l] + (1-self.beta2)*(g[l]*g[l])
            self.mhat = self.m[l]/(1-self.beta1**self.t)
            self.shat = self.s[l]/(1-self.beta2**self.t)
            theta2[l] = theta[l]-self.lr*((self.mhat/(pow(self.shat,0.5)+self.epislon))+self.decay*theta[l])
        return theta2*1
class ConvNet(object):
    def __init__(self):
        channel = 1
        '''
        2层卷积，2层池化，3层全连接'''
        self.filters = [cp.random.normal(0,0.1,(6, 5, 5, channel))] 
        self.filters_biases = [cp.zeros((6,channel))]
        self.filters.append(cp.random.normal(0,0.1,(16, 5, 5, 6)) )
        self.filters_biases.append(cp.zeros((16,channel)))
        self.weights = [cp.random.normal(0,0.1,(120,400))]
        self.weights.append(cp.random.normal(0,0.1,(84,120)))
        self.weights.append(cp.random.normal(0,0.1,(10,84)))
        self.biases = [0.1*cp.ones((120,1))]
        self.biases.append(0.1*cp.ones((84,1)))
        self.biases.append(0.1*cp.ones((10,1)))
        self.Adam_b= Adam(self.biases)
        self.Adam_w = Adam(self.weights)
        self.Adam_ft = Adam(self.filters)
        self.Adam_fb = Adam(self.filters_biases)
    def Backward(self, x,y,activate,dactivate,evaluate=False):
        #第一层卷积
        conv_all=[]
        pool_all=[]
        pool_loc=[]
        conv1 = add_bias(conv(x, self.filters[0]),self.filters_biases[0] )
        relu1 = activate(conv1)
        pool1, pool1_max_locate = pool(relu1)#num*14*14*6
        conv2 = add_bias(conv(pool1, self.filters[1]), self.filters_biases[1])
        relu2 = activate(conv2)
        pool2, pool2_max_locate = pool(relu2) #num*5*5*16
        straight_input = pool2.reshape(pool2.shape[0],pool2.shape[1] * pool2.shape[2] * pool2.shape[3])
        zm=[]
        active=[straight_input.T*1]
        for l in range(3):
            full_connect_z = cp.dot(self.weights[l], active[l]) + self.biases[l]
            zm.append(full_connect_z)
            if l == 2:
                full_connect_a = softmax(full_connect_z)
                active.append(full_connect_a)
            else:
                full_connect_a = activate(full_connect_z)          
                active.append(full_connect_a)
        delta_full=[]
        nabla_w=[cp.zeros(w_s.shape) for w_s in self.weights]
        nabla_bias=[cp.zeros(b_s.shape) for b_s in self.biases]
        for l in range(1, 4):
            if l == 1:
                delta = active[-1]-cp.squeeze(y.T)
                delta_full.append(delta)
            else:
                delta = cp.dot(self.weights[-l+1].transpose(), delta_full[-1]) * dactivate(zm[-l])
                delta_full.append(delta)
            nabla_w[-l] = (cp.dot(delta,(active[-l-1].T)))/(cp.shape(x)[0])
            nabla_bias[-l] = (cp.sum(delta,axis=1,keepdims=True)/(cp.shape(x)[0]))
            
        delta_full.append(cp.dot(self.weights[0].transpose(), delta_full[-1]))
        delta_pool2 = (delta_full[-1].T).reshape(pool2.shape)#(100, 5, 5, 16)
        delta_conv2 = pool_delta_error_bp(delta_pool2, pool2_max_locate) * dactivate(conv2)
        delta_pool1 = conv(padding(delta_conv2, self.filters[1].shape[1]-1), rotate180(self.filters[1]).swapaxes(0,3))* dactivate(pool1)
        delta_conv1 = pool_delta_error_bp(delta_pool1, pool1_max_locate) * dactivate(conv1)
        nabla_filters1 = reverse_conv2d(pool1,delta_conv2,rotate=False,invert = True) 
        nabla_filters0 = reverse_conv2d(x,delta_conv1,rotate=False,invert = True)
        nabla_filters_biases1 = conv_cal_b(delta_conv2)
        nabla_filters_biases0 = conv_cal_b(delta_conv1)
        nabla_f = [nabla_filters0, nabla_filters1]
        nabla_fb = [nabla_filters_biases0, nabla_filters_biases1]

        if evaluate == True:
            return active[-1]
        if evaluate == False:
            return nabla_w, nabla_bias, nabla_f, nabla_fb
        
    def evaluate(self, test_data1,test_label1,activate,dactivate):
        # 获得预测结果a:10*batch_size
        #testlabel:batch_size*10*1
        test_data = test_data1*1 
        test_label=test_label1*1
        data,label = mini_batch_generate(500,test_data,test_label)
        accuracy=[]
        for i in range(data.shape[0]):
            a=self.Backward(data[i],label[i],activate,dactivate,evaluate=True)
            max0=cp.argmax(a,axis=0).reshape(a.shape[1],1)
            max1=cp.argmax(label[i],axis=1)
            acc=((cp.sum((max0-max1) == 0))/(data[i].shape[0]))
            accuracy.append(acc)
        return cp.average(accuracy)
    def update_mini_batch(self,  eta, mini_batch_size):
        '''通过一个batch的数据对神经网络参数进行更新
        需要先求这个batch中每张图片的误差反向传播求得的权重梯度以及偏置梯度'''
        data_x=train_image*1
        label_x=train_label0*1
        data,label = mini_batch_generate(mini_batch_size,data_x,label_x)
        for j in range(data.shape[0]):
            nabla_b = [cp.zeros(b.shape) for b in self.biases]
            nabla_w = [cp.zeros(w.shape) for w in self.weights]
        
            nabla_f = [cp.zeros(f.shape) for f in self.filters]
            nabla_fb = [cp.zeros(fb.shape) for fb in self.filters_biases]

            delta_nabla_w, delta_nabla_b, delta_nabla_f, delta_nabla_fb = self.Backward(data[j],label[j],relu,drelu,evaluate=False)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_f = [nf+dnf for nf, dnf in zip(nabla_f, delta_nabla_f)]
            nabla_fb = [nfb + dnfb for nfb, dnfb in zip(nabla_fb, delta_nabla_fb)]
            self.weights = [w-(eta/mini_batch_size)*nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b-(eta/mini_batch_size)*nb for b, nb in zip(self.biases, nabla_b)]
            self.filters = [f-(eta/mini_batch_size)*nf for f, nf in zip(self.filters, nabla_f)]
            self.filters_biases = [fb-(eta/mini_batch_size)*nfb for fb, nfb in zip(self.filters_biases, nabla_fb)]
            print('\r'+str(j)+'/'+str(int(data.shape[0])),end='')
    def adam_update(self,lr,mini_batch_size,activate,dactivate,train_data_x,train_label_x):
        data_x=train_data_x*1
        label_x=train_label_x*1
        data,label = mini_batch_generate(mini_batch_size,data_x,label_x)
        for j in range(data.shape[0]):
            delta_nabla_w, delta_nabla_b, delta_nabla_f, delta_nabla_fb =  self.Backward(data[j],label[j],activate,dactivate,evaluate=False)
            self.weights = self.Adam_w.New_theta(self.weights,delta_nabla_w,lr)
            self.biases =  self.Adam_b.New_theta(self.biases,delta_nabla_b,lr)
            self.filters = self.Adam_ft.New_theta(self.filters,delta_nabla_f,lr)
            self.filters_biases = self.Adam_fb.New_theta(self.filters_biases,delta_nabla_fb,lr)
            print('\r'+str(j)+'/'+str(int(data.shape[0])),end='')
    def SGD(self,mini_batch_size,epoch,lr0,activate,dactivate):
        acc1_=[]
        for i in range(epoch):
            train_labelt=train_label0*1#改参数
            train_datat=train_image*1
            lr = divi_(lr0,i,40)
            self.adam_update(lr,mini_batch_size,activate,dactivate,train_datat,train_labelt)
#             self.update_mini_batch(lr, mini_batch_size)
            print ("Epoch %s training complete" % i)
            acc1 = self.evaluate(test_image,test_label0,activate,dactivate)
            print("the test Accuracy for task0 is:{} %".format((acc1)*100))
            acc1_.append(acc1*100)
        return acc1_
    

In [ ]:
net = ConvNet()
xx=net.SGD(100,120,0.001,relu,drelu)

599/600Epoch 0 training complete
the test Accuracy for task0 is:97.33000000000001 %
55/600